In [220]:
import requests
import os
import pandas as pd

In [221]:
df=pd.read_csv('data.csv')
df.head()

,name,latitude,longitude,free_bikes,empty_slots,total_bike_cap
0,East End Library,37.539779,-77.411460,0,6,6
1,Sydney Park,37.546968,-77.456599,0,16,16
2,Broad & Harrison,37.551472,-77.451937,0,19,19
3,Broad & Lombardy,37.553716,-77.455681,0,23,23
4,Broad Rock Library,37.483291,-77.479390,0,10,10


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [233]:
api_key = os.environ['foursquareapi']
latitude = ""
longitude = ''
radius = str(1000)
categories = '11124,13003'
headers = {"Accept": "application/json"}
headers['Authorization'] = api_key
foursquaredata = pd.DataFrame()

for index, row in df.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    url = f"https://api.foursquare.com/v3/places/search?ll={latitude},{longitude}&radius={radius}&categories={categories}&fields=name,rating,location,popularity,distance,categories"
    result = requests.get(url, headers=headers)
    data = result.json()
    result_df = pd.json_normalize(data.get('results', []))
    if not result_df.empty:
        result_df['station'] = row['name']
        foursquaredata = pd.concat([foursquaredata, result_df], ignore_index=True)

print(foursquaredata['station'].unique())  # Check unique stations in the final DataFrame


['East End Library' 'Sydney Park' 'Broad & Harrison' 'Broad & Lombardy'
 'Broad Rock Library' 'Abner Clay Park' 'Center Stage'
 'Main Street Station' 'Carter Jones Park' 'Kanawha Plaza' 'Monroe Park'
 'Battery Park' 'City Hall' 'Blackwell' 'Fairmont' 'Science Museum'
 'Browns Island' 'Jefferson Ave' 'Chimborazo Park' 'Canal Walk'
 'Pleasants Park-Oregon Hill' 'North Avenue Library'
 'Petronius Jones Park-Randolph' 'Main Library']


In [235]:
foursquaredata['station'].unique()

array(['East End Library', 'Sydney Park', 'Broad & Harrison',
       'Broad & Lombardy', 'Broad Rock Library', 'Abner Clay Park',
       'Center Stage', 'Main Street Station', 'Carter Jones Park',
       'Kanawha Plaza', 'Monroe Park', 'Battery Park', 'City Hall',
       'Blackwell', 'Fairmont', 'Science Museum', 'Browns Island',
       'Jefferson Ave', 'Chimborazo Park', 'Canal Walk',
       'Pleasants Park-Oregon Hill', 'North Avenue Library',
       'Petronius Jones Park-Randolph', 'Main Library'], dtype=object)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [236]:
foursquareparsed = foursquaredata[['station','distance','name','location.address','rating','popularity','categories']]
foursquareparsed.loc[:, 'categories']= foursquareparsed['categories'].apply(lambda x: [cat['name'] for cat in x])

In [238]:
foursquareparsed.loc[:, 'categories'] = foursquareparsed['categories'].apply(lambda x: 'bar' if any('bar' in category.lower() for category in x) or any('pub' in category.lower() for category in x) or any('restaurant' in category.lower() for category in x) else 'office')

### Dealing with duplicates

In [239]:
namecount = foursquareparsed.groupby(['name', 'station']).size().reset_index(name='count')
namecount[namecount['count']>1].sort_values(by='count',ascending=False)
foursquareparsed=foursquareparsed.drop_duplicates(subset='name')

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [241]:
yelp_api_key = os.environ['yelpapi']
latitude = ""
longitude = ''
radius = str(1000)
categories = 'bars,offices'
headers = {"Accept": "application/json"}
headers['Authorization'] = f"Bearer {yelp_api_key}"
yelpdataframes = []
for index, row in df.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    url = f"https://api.yelp.com/v3/businesses/search?latitude={latitude}&longitude={longitude}&radius={radius}&categories={categories}"
    result = requests.get(url, headers=headers)
    print(f"Error: {result.status_code}, {result.text}")
    data = result.json()
    result_df = pd.json_normalize(data.get('businesses', []))
    if not result_df.empty:
        result_df['station'] = row['name']
        yelpdataframes.append(result_df)
yelpdata = pd.concat(yelpdataframes, ignore_index=True) if yelpdataframes else pd.DataFrame()


Error: 429, {"error": {"code": "ACCESS_LIMIT_REACHED", "description": "You've reached the access limit for this client. See instructions for requesting a higher access limit at https://docs.developer.yelp.com/docs/fusion-rate-limiting"}}
Error: 429, {"error": {"code": "ACCESS_LIMIT_REACHED", "description": "You've reached the access limit for this client. See instructions for requesting a higher access limit at https://docs.developer.yelp.com/docs/fusion-rate-limiting"}}
Error: 429, {"error": {"code": "ACCESS_LIMIT_REACHED", "description": "You've reached the access limit for this client. See instructions for requesting a higher access limit at https://docs.developer.yelp.com/docs/fusion-rate-limiting"}}
Error: 429, {"error": {"code": "ACCESS_LIMIT_REACHED", "description": "You've reached the access limit for this client. See instructions for requesting a higher access limit at https://docs.developer.yelp.com/docs/fusion-rate-limiting"}}
Error: 429, {"error": {"code": "ACCESS_LIMIT_REA

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [227]:
yelpdata['station'].unique()

KeyError: 'station'

In [171]:
yelpparsed = yelpdata[['station','distance','name','location.address1','rating']]
yelpparsed=yelpparsed.rename(columns={'location.address1': 'location.address'})


### Dealing with duplicates

In [172]:
namecount = yelpparsed.groupby(['name', 'station']).size().reset_index(name='count')
namecount[namecount['count']>1].sort_values(by='count',ascending=False)
yelpparsed=yelpparsed.drop_duplicates(subset='name')


Dropping distance as a column since both had duplicates with inaccurate data based on distance.

In [176]:
mergeddf=pd.merge(yelpparsed,foursquareparsed,on=['name','location.address'],how='inner',suffixes=('yelp','4sq'))
mergeddf.head()
mergeddf = mergeddf[['stationyelp','name','location.address','ratingyelp','rating4sq','categories4sq','categoriesyelp']]

Converting yelp rating to a ten scale for equal review

In [177]:
mergeddf['ratingyelp']=mergeddf[['ratingyelp']]*2
mergeddf.head()

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Foursquare provided more complete data because they provide clear documentation around categories and filtering. Yelp's lack of category definitions means that there is more uncertainty around what you are able to pull. 

Get the top 10 restaurants according to their rating

Yelp top 10

In [178]:
yelptop10 = mergeddf[mergeddf['categoriesyelp']=='bar'].sort_values(by='ratingyelp', ascending=False).reset_index(drop=True).head(10)
yelptop10

Foursquare top 10

In [179]:
foursqtop10 = mergeddf[mergeddf['categories4sq']=='bar'].sort_values(by='rating4sq', ascending=False).reset_index(drop=True).head(10)
foursqtop10